In [1]:
import pandas as pd
import numpy as np
import math
import pickle

from scipy import stats
import scipy.io
from scipy.spatial.distance import pdist
from scipy.linalg import cholesky
from scipy.io import loadmat

import matlab.engine as engi
import matlab as mat

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,roc_auc_score,recall_score,precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from pyearth import Earth

from src import SMOTE
from src import CFS
from src import metrices_V2 as metrices

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path

import matplotlib.pyplot as plt

# Data Load and other util function

In [2]:
def load_data(project):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    commit_guru_path = 'data/commit_guru/' + project + '.csv'
    understand_df = pd.read_csv(understand_path)
    understand_df = understand_df.dropna(axis = 1,how='all')
    cols_list = understand_df.columns.values.tolist()
#     print(cols_list)
    for item in ['Kind', 'Name','commit_hash', 'Bugs']:
        if item in cols_list:
            cols_list.remove(item)
            cols_list.insert(0,item)
    understand_df = understand_df[cols_list]
    commit_guru_df = pd.read_csv(commit_guru_path)
    cols = understand_df.columns.tolist()
    
    commit_guru_df = commit_guru_df.drop(labels = ['parent_hashes','author_name','author_name',
                                                   'author_email','fileschanged','author_date',
                                                   'author_date_unix_timestamp', 'commit_message',
                                                  'classification', 'fix', 'contains_bug','fixes',],axis=1)

    understand_df = understand_df.drop_duplicates(cols[4:len(cols)])
    df = understand_df
    cols = df.columns.tolist()
    cols = cols[1:] + [cols[0]]
    df = df[cols]
    for item in ['Kind', 'Name','commit_hash']:
        if item in cols:
            df = df.drop(labels = [item],axis=1)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    cols = X.columns
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    return X,y

def apply_smote(df):
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def apply_cfs(df):
        y = df.Bugs.values
        X = df.drop(labels = ['Bugs'],axis = 1)
        X = X.values
        selected_cols = CFS.cfs(X,y)
        cols = df.columns[[selected_cols]].tolist()
        cols.append('Bugs')
        return df[cols],cols

# Function to build predictor and get performance

## Without CFS

In [3]:
def run_self(project):
    X,y = load_data(project)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
    loc = X_test.CountLineCode
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf = RandomForestClassifier()
    clf.fit(X_train,y_train)
    predicted = clf.predict(X_test)
    abcd = metrices.measures(y_test,predicted,loc)
    pf = abcd.get_pf()
    recall = abcd.calculate_recall()
    precision = abcd.calculate_precision()
    f1 = abcd.calculate_f1_score()
    g_score = abcd.get_g_score()
    pci_20 = abcd.get_pci_20()
    ifa = abcd.get_ifa()
    try:
        auc = roc_auc_score(y_test, predicted)
    except:
        auc = 0
    print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa

## With CFS

In [20]:
def run_self_CFS(project):
    X,y = load_data(project)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
    loc = X_test.CountLineCode
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    df_smote,cols = apply_cfs(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf = RandomForestClassifier()
    clf.fit(X_train,y_train)
    predicted = clf.predict(X_test[cols[:-1]])
    abcd = metrices.measures(y_test,predicted,loc)
    pf = abcd.get_pf()
    recall = abcd.calculate_recall()
    precision = abcd.calculate_precision()
    f1 = abcd.calculate_f1_score()
    g_score = abcd.get_g_score()
    pci_20 = abcd.get_pci_20()
    ifa = abcd.get_ifa()
    try:
        auc = roc_auc_score(y_test, predicted)
    except:
        auc = 0
    print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa

# Getting List of projects

In [21]:
proj_df = pd.read_csv('projects.csv')
projects = proj_df.repo_name.tolist()

# Run the experiment

In [ ]:
precision_list = {}
recall_list = {}
pf_list = {}
f1_list = {}
g_list = {}
auc_list = {}
pci_20_list = {}
ifa_list = {}
for project in projects:
    try:
        if project == '.DS_Store':
            continue
    #     if project != 'guice':
    #         continue
        print("+++++++++++++++++   "  + project + "  +++++++++++++++++")
        recall,precision,pf,f1,g_score,auc,pci_20,ifa = run_self_CFS(project)
        recall_list[project] = recall
        precision_list[project] = precision
        pf_list[project] = pf
        f1_list[project] = f1
        g_list[project] = g_score
        auc_list[project] = auc
        pci_20_list[project] = pci_20
        ifa_list[project] = ifa
    except Exception as e:
        print(e)
        continue
final_result = {}
final_result['precision'] = precision_list
final_result['recall'] = recall_list
final_result['pf'] = pf_list
final_result['f1'] = f1_list
final_result['g'] = g_list
final_result['auc'] = auc_list
final_result['pci_20'] = pci_20_list
final_result['ifa'] = ifa_list
with open('results/Performance/product_1000_CFS.pkl', 'wb') as handle:
    pickle.dump(final_result, handle, protocol=pickle.HIGHEST_PROTOCOL)

+++++++++++++++++   org.alloytools.alloy  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.92      0.95      0.93       138
           1       0.96      0.93      0.94       168

    accuracy                           0.94       306
   macro avg       0.94      0.94      0.94       306
weighted avg       0.94      0.94      0.94       306

+++++++++++++++++   qpython  +++++++++++++++++
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       164

    accuracy                           1.00       164
   macro avg       1.00      1.00      1.00       164
weighted avg       1.00      1.00      1.00       164

+++++++++++++++++   friendlychat-android  +++++++++++++++++
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00         7

    accuracy                           1.00        10
   macro avg   

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       656

    accuracy                           1.00       656
   macro avg       1.00      1.00      1.00       656
weighted avg       1.00      1.00      1.00       656

+++++++++++++++++   DataSphereStudio  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.86      0.91      0.88        87
           1       0.20      0.13      0.16        15

    accuracy                           0.79       102
   macro avg       0.53      0.52      0.52       102
weighted avg       0.76      0.79      0.78       102

+++++++++++++++++   mapstruct-examples  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.93      0.90      0.92        30
           1       0.00      0.00      0.00         2

    accuracy                           0.84        32
   macro avg       0.47      0.45      0.46        32
weighted avg  

              precision    recall  f1-score   support

           0       0.86      0.86      0.86        44
           1       0.60      0.60      0.60        15

    accuracy                           0.80        59
   macro avg       0.73      0.73      0.73        59
weighted avg       0.80      0.80      0.80        59

+++++++++++++++++   project-examples  +++++++++++++++++
float division by zero
+++++++++++++++++   rx-preferences  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.62      0.62      0.62         8
           1       0.40      0.40      0.40         5

    accuracy                           0.54        13
   macro avg       0.51      0.51      0.51        13
weighted avg       0.54      0.54      0.54        13

+++++++++++++++++   crystal-range-seekbar  +++++++++++++++++
              precision    recall  f1-score   support

         0.0       1.00      0.33      0.50        18
         1.0       0.00      0.00      0.00

              precision    recall  f1-score   support

           0       0.91      0.84      0.87        49
           1       0.27      0.43      0.33         7

    accuracy                           0.79        56
   macro avg       0.59      0.63      0.60        56
weighted avg       0.83      0.79      0.80        56

+++++++++++++++++   ViewSupport  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.50      0.80      0.62         5
           1       0.00      0.00      0.00         4

    accuracy                           0.44         9
   macro avg       0.25      0.40      0.31         9
weighted avg       0.28      0.44      0.34         9

+++++++++++++++++   Isometric  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.38      0.43      0.40         7
           1       0.33      0.29      0.31         7

    accuracy                           0.36        14
   macro avg       0.35     

              precision    recall  f1-score   support

           0       0.81      0.70      0.75        30
           1       0.10      0.17      0.12         6

    accuracy                           0.61        36
   macro avg       0.45      0.43      0.44        36
weighted avg       0.69      0.61      0.65        36

+++++++++++++++++   XUpdate  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.80      0.76      0.78        59
           1       0.22      0.27      0.24        15

    accuracy                           0.66        74
   macro avg       0.51      0.51      0.51        74
weighted avg       0.69      0.66      0.67        74

+++++++++++++++++   jenkins-hipchat-plugin  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.57      0.40      0.47        10
           1       0.25      0.40      0.31         5

    accuracy                           0.40        15
   macro avg       

...

              precision    recall  f1-score   support

           0       0.77      0.82      0.80       914
           1       0.43      0.36      0.39       342

    accuracy                           0.69      1256
   macro avg       0.60      0.59      0.59      1256
weighted avg       0.68      0.69      0.69      1256

+++++++++++++++++   rxjava-jdbc  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.70      0.78      0.74       237
           1       0.59      0.48      0.53       155

    accuracy                           0.66       392
   macro avg       0.64      0.63      0.63       392
weighted avg       0.66      0.66      0.66       392

+++++++++++++++++   aws-sdk-android  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.86      0.91      0.88      4018
           1       0.36      0.24      0.29       815

    accuracy                           0.80      4833
   macro avg       0.6

              precision    recall  f1-score   support

           0       0.82      0.77      0.79       126
           1       0.50      0.57      0.53        51

    accuracy                           0.71       177
   macro avg       0.66      0.67      0.66       177
weighted avg       0.72      0.71      0.72       177

+++++++++++++++++   gitiles  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.66      0.74      0.70       291
           1       0.56      0.47      0.51       208

    accuracy                           0.63       499
   macro avg       0.61      0.60      0.61       499
weighted avg       0.62      0.63      0.62       499

+++++++++++++++++   extentreports-java  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.48      0.45      0.46       150
           1       0.58      0.61      0.60       187

    accuracy                           0.54       337
   macro avg       0.53

              precision    recall  f1-score   support

           0       0.41      0.43      0.42        83
           1       0.67      0.65      0.66       147

    accuracy                           0.57       230
   macro avg       0.54      0.54      0.54       230
weighted avg       0.58      0.57      0.57       230

+++++++++++++++++   google-java-format  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.68      0.70      0.69       244
           1       0.49      0.47      0.48       152

    accuracy                           0.61       396
   macro avg       0.59      0.58      0.58       396
weighted avg       0.61      0.61      0.61       396

+++++++++++++++++   fixflow  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.79      0.84      0.82      1529
           1       0.33      0.26      0.29       454

    accuracy                           0.71      1983
   macro avg       0.56

              precision    recall  f1-score   support

           0       0.62      0.57      0.59       383
           1       0.53      0.58      0.55       318

    accuracy                           0.57       701
   macro avg       0.57      0.57      0.57       701
weighted avg       0.58      0.57      0.57       701

+++++++++++++++++   pippo  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.75      0.85      0.80       526
           1       0.37      0.23      0.28       197

    accuracy                           0.68       723
   macro avg       0.56      0.54      0.54       723
weighted avg       0.64      0.68      0.66       723

+++++++++++++++++   SeLion  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.60      0.60      0.60       367
           1       0.61      0.61      0.61       369

    accuracy                           0.60       736
   macro avg       0.60      0.60    

              precision    recall  f1-score   support

           0       0.68      0.69      0.68       266
           1       0.60      0.58      0.59       212

    accuracy                           0.64       478
   macro avg       0.64      0.64      0.64       478
weighted avg       0.64      0.64      0.64       478

+++++++++++++++++   programming  +++++++++++++++++
index 1 is out of bounds for axis 0 with size 1
+++++++++++++++++   OpenRTS  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.68      0.66      0.67       640
           1       0.78      0.79      0.78       959

    accuracy                           0.74      1599
   macro avg       0.73      0.72      0.72      1599
weighted avg       0.74      0.74      0.74      1599

+++++++++++++++++   spectator  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.64      0.62      0.63       358
           1       0.67      0.68      0.6